In [ ]:
# Update with your own model name

model_name = 'sagemaker-xgboost-2020-06-09-08-33-24-782'

In [ ]:
production_variants = [
        {
            'VariantName': 'variant-1',
            'ModelName': model_name,
            'InitialInstanceCount': 2,
            'InitialVariantWeight': 1,
            'InstanceType': 'ml.m5.large'
        }
]

In [ ]:
import boto3
import pprint

sm = boto3.client('sagemaker')

In [ ]:
endpoint_config_name = 'xgboost-one-model-epc'

In [ ]:
response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants,
    Tags=[{'Key': 'Name','Value': endpoint_config_name},
          {'Key': 'Algorithm','Value': 'xgboost'}]
)
pprint.pprint(response)

In [ ]:
response = sm.describe_endpoint_config(EndpointConfigName=endpoint_config_name)

pprint.pprint(response)

In [ ]:
endpoint_name = 'xgboost-one-model-ep'

In [ ]:
response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    Tags=[{'Key': 'Name','Value': endpoint_name},
          {'Key': 'Algorithm','Value': 'xgboost'},
          {'Key': 'Environment','Value': 'development'}]
)

In [ ]:
response = sm.describe_endpoint(EndpointName=endpoint_name)

pprint.pprint(response)

In [ ]:
app = boto3.client('application-autoscaling')

In [ ]:
response = app.register_scalable_target(
    ServiceNamespace='sagemaker',
    ResourceId='endpoint/xgboost-one-model-ep/variant/variant-1',
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=2,
    MaxCapacity=10)

In [ ]:
policy_name = 'xgboost-scaling-policy'

response = app.put_scaling_policy(
    PolicyName=policy_name,
    ServiceNamespace='sagemaker',
    ResourceId='endpoint/xgboost-one-model-ep/variant/variant-1',
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 1000.0,
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'
        },
        "ScaleInCooldown": 60,
        "ScaleOutCooldown": 60
    }
)

In [ ]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98'

In [ ]:
smrt = boto3.Session().client(service_name='runtime.sagemaker') 

while True:
    smrt.invoke_endpoint(EndpointName=endpoint_name, 
                         ContentType='text/csv', 
                         Body=test_sample)

In [ ]:
app.delete_scaling_policy(
    PolicyName=policy_name,
    ServiceNamespace='sagemaker',
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    ResourceId='endpoint/xgboost-one-model-ep/variant/variant-1')

In [ ]:
sm.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)